# Data Collection

The original dataset, US Domestic Flights Delay Prediction (2013 - 2018) (Source: [Kaggle](https://www.kaggle.com/datasets/gabrielluizone/us-domestic-flights-delay-prediction-2013-2018)), is provided as a zip archive of 1.54 GB. When decompressed, the archive contains 60 files with a total size of 13.6 GB. Each file corresponds to one month of data, starting from January 2014 and ending in December 2018.

Steps for collecting and cleaning the data:

*	Loading the first file
*	Evaluating the data’s size and quality
*	Defining the specific data needed for the project
*	Developing a process for data organization and cleaning
*	Testing the code on one month’s data file
*	Applying the code to all files
*	Accumulating the complete dataset


The archive file (csv_flight.zip) was downloaded and saved locally.

## Loading the first file

In [1]:
import zipfile
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
from dateutil import tz
from time import time
import pickle

os.chdir('/Users/a.kholodov/Documents/02. Personal/20. Education/50. Universities/Springboard/Springboard_git/Springboard _repo/CS2-flights-delay-REPO')

# pd.set_option('display.max_rows', 130)
# pd.set_option('display.width', 200)

# Loading timezones for IATA codes of airports
IATAtz_df = pd.read_csv('https://raw.githubusercontent.com/hroptatyr/dateutils/tzmaps/iata.tzmap', 
                        sep = '\t', 
                        index_col=0, 
                        header=None)

# Dictionary with IATA codes as keys and timezones as values
IATAtz = IATAtz_df.to_dict('dict')[1]
del(IATAtz_df)

In [2]:
source_zip_file = 'data/interim/csv_flight.zip'
data_file = 'csv_flight/report_2014_1.csv'

# reading the first file to evaluate the data
with zipfile.ZipFile(source_zip_file) as zip_source:
    with zip_source.open(data_file) as data_file:
        flights_2014_1 = pd.read_csv(data_file, low_memory=False)

## Data size

In [ ]:
print(flights_2014_1.shape)
print(flights_2014_1.info())

The data for one month contains 471,949 rows and 110 columns, with a total memory size of 396 MB. The estimated size of the entire dataset, without reorganization or cleaning, may exceed 23 GB, which could be challenging to process locally. Therefore, one of the goals of data preparation will be to reduce the dataset size without compromising quality.

## Data structure and check
### Date of the flight

In [ ]:
flights_2014_1.iloc[:,:6].head()

0. **Year**
    * **Description:** Year
    * **Data type:** int16
    * **Drop**
    * **Comment:** We have a flight date which includes this data

1. **Quarter**  
    * **Description:** Quarter (1-4)  
    * **Data type:** int8
    * **Drop**
    * **Comment:** We have a flight date which includes this data

2. **Month**  
    * **Description:** Month (1-12)  
    * **Data type:** int8
    * **Drop**
    * **Comment:** We have a flight date which includes this data

3. **DayofMonth**  
    * **Description:** Day of month  
    * **Data type:** int8
    * **Drop**
    * **Comment:** We have a flight date which includes this data

4. **DayOfWeek**  
    * **Description:** Day of week  
    * **Data type:** int8
    * **Drop**
    * **Comment:** We have a flight date which includes this data

5. **FlightDate**  
    * **Description:** Flight Date (yyyymmdd)
    * **Data type:** datetime
    * **Keep**

Values check for Date of the flight: the data accurate, there is no outliers or NA.

In [ ]:
flights_2014_1['FlightDate'].describe()

In [ ]:
flights_2014_1['FlightDate'].value_counts()

But we need to change the data type during the data transformation from current type to datetime.

In [ ]:
flights_2014_1['FlightDate'].info()

In [ ]:
flights_2014_1['FlightDate'].isna().sum()

###  Airline's and flight's details 

In [ ]:
flights_2014_1.iloc[:, 6:10].head()


6. **Reporting_Airline**
    * **Description:**  Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years. 
    * **Data type:** category 
    * **Keep**
    * **Comment:** I decided to keep this field and drop DOT_ID_Reporting_Airline and IATA_CODE_Reporting_Airline because it is declared as unique and recomended by data provider as such | 

7. **DOT_ID_Reporting_Airline**
    * **Description** An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
    * **Drop**
    * **Comment:** Not unique
    
8. **IATA_CODE_Reporting_Airline**
    * **Description:** Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
    * **Drop**
    * **Comment:** Not unique

9. **Tail_Number**
    * **Description:** Tail Number
    * **Drop**
    * **Comment:** Irrelevant to the purposes of the project

10. **Flight_Number_Reporting_Airline**
    * **Description:** Flight Number
    * **Data type:** int16
    * **Keep**
    * **Comment:** Unique number of the flight at a specific day/time

In [ ]:
flights_2014_1.iloc[:, 6].unique()

In [ ]:
flights_2014_1['Flight_Number_Reporting_Airline'].describe()

In [ ]:
flights_2014_1[['Reporting_Airline', 'Flight_Number_Reporting_Airline']].info()

In [ ]:
flights_2014_1[['Reporting_Airline', 'Flight_Number_Reporting_Airline']].isna().sum()

The data in the fields Reporting_Airline and Flight_Number_Reporting_Airline is accurate, there is no outliers or NA, But I need to change the data type during the data transformation.

###  Origin and Destination detailes

In [ ]:
DestFields = ['DestAirportID', 
                'DestAirportSeqID', 
                'DestCityMarketID',
                'Dest',
                'DestCityName',
                'DestState',
                'DestStateFips',
                'DestStateName',
                'DestWac']
flights_2014_1.loc[:5, DestFields]

In [ ]:
OriginFields = ['OriginAirportID', 
                'OriginAirportSeqID', 
                'OriginCityMarketID',
                'Origin',
                'OriginCityName',
                'OriginState',
                'OriginStateFips',
                'OriginStateName',
                'OriginWac']
flights_2014_1.loc[:5, OriginFields]

Origin and Destination data has the same structure so I will treat it the same way.

11.	**OriginAirportID / 20. DestAirportID**  
    * **Description:** Origin/Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.  
    * **Data type:** category  
    * **Keep**  
    * **Comment:** I decided to keep these codes because of its uniqueness, assuared by data provider.  
    
12.	**OriginAirportSeqID / 21. DestAirportSeqID**  
    * **Description:** Origin/Destination Airport, Airport Sequence ID. An identification number assigned by US DOT to identify a unique airport at a given point of time. Airport attributes, such as airport name or coordinates, may change over time.
    * **Drop**
13.	**OriginCityMarketID / 22. DestCityMarketID**
    * **Description:** Origin/Destination Airport, City Market ID. City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.
    * **Drop**
14.	**Origin / 23. Dest**
    * **Description:** Origin/Destination Airport
    * **Data type:** category
    * **Keep**
    * **Comment:** This core is IATA code of the airport, which is represented in most traveling documents. It could be useful in the model.
15.	**OriginCityName / 24. DestCityName**
    * **Description:** Origin/Destination Airport, City Name
    * **Drop**
16.	**OriginState / 25. DestState**
    * **Description:** Origin/Destination Airport, State Code
    * **Drop**
17.	**OriginStateFips / 26. DestStateFips**
    * **Description:** Origin/Destination Airport, State Fips
    * **Drop**
18.	**OriginStateName / 27. DestStateName**
    * **Description:** Origin/Destination Airport, State Name
    * **Drop**
19.	**OriginWac / 28. DestWac**
    * **Description:** Origin/Destination Airport, World Area Code
    * **Drop**

In [ ]:
flights_2014_1.loc[:, 'OriginAirportID'].unique()

In [ ]:
flights_2014_1.loc[:, 'DestAirportID'].unique()

In [ ]:
flights_2014_1.loc[:, 'Origin'].unique()

In [ ]:
flights_2014_1.loc[:, 'Dest'].unique()

In [ ]:
# Check for equal number of Origin Airport IDs and Origin (IATA codes)
print('Number of unique OriginAirport IDs:', flights_2014_1.loc[:, 'OriginAirportID'].nunique(),
      '\nNumber of unique Origin codes:', flights_2014_1.loc[:, 'Origin'].nunique())

In [ ]:
# Check for equal number of Origin Airport IDs and Origin (IATA codes)
print('Number of unique DestAirport IDs:', flights_2014_1.loc[:, 'DestAirportID'].nunique(),
      '\nNumber of unique Destination codes:', flights_2014_1.loc[:, 'Dest'].nunique())

In [ ]:
# Check for NA-values in the fields, I suppose to keep
flights_2014_1[['OriginAirportID', 'Origin', 'DestAirportID', 'Dest']].isna().sum()

The numbers of unique code for OriginalAirportID/DestAirportID and Origin/Dest are equal, and they don't have NA-values.

###  Departure and Arrival times (scheduled and actual)

In [ ]:
# Sample of the block of data for departure times
DepFields = ['CRSDepTime', 
             'DepTime',
             'DepDelay',
             'DepDelayMinutes',
             'DepDel15',
             'DepartureDelayGroups',
             'DepTimeBlk']
flights_2014_1.loc[:5, DepFields]

In [ ]:
# Sample of the block of data for arrival times
ArrFields = ['CRSArrTime', 
             'ArrTime',
             'ArrDelay',
             'ArrDelayMinutes',
             'ArrDel15',
             'ArrivalDelayGroups',
             'ArrTimeBlk']
flights_2014_1.loc[:5, ArrFields]

Two groups of data (for Departure and Arrival time) have the same stricture and similar meaning (with difference in departure or arrival), so I group and describe them togethher.

All time date in dataset is integer in format HHMM. I think for the further analysis it's worth to convert it into the number of minutes from the start of the day.

29.	**CRSDepTime  40. CRSArrTime** 
    * **Description:** CRS Departure/Arrival Time (local time: hhmm)
    * **Data type:** float
    * **Keep**
    * **Comment:** CRS (Computer Reservation System) represents the scuduled time for the flight. I decided to keep this data at this stage becuase it's not clear yet which will be the drivers of the future model - time or categorical time blocks, such as DepTimeBlk or ArrTimeBlk. I suppose to make this decision later. Needs to conver to the number of minutes.
30.	**DepTime  41. ArrTime**  
    * **Description:** Actual Departure/Arrival Time (local time: hhmm)
    * **Data type:** float
    * **Keep**
    * **Comment:** Needs to conver to the number of minutes.
31.	**DepDelay  42. ArrDelay**  
    * **Description:** Difference in minutes between scheduled and actual departure/arrival time. Early departures/arrival show negative numbers.
    * **Data type:** float
    * **Keep**
32.	**DepDelayMinutes  43. ArrDelayMinutes**
    * **Description:** Difference in minutes between scheduled and actual departure/arrival time. Early departures/arrival set to 0.
    * **Drop**
    * **Comment:** This data dublicates partially the field DepDelay and ArrDelay with only differency that this data doesnt't show the negative values - departures or arrivals earlier
33.	**DepDel15  44. ArrDel15**
    * **Description:** Departure/Arrival Delay Indicator, 15 Minutes or More (1=Yes)
    * **Drop**
    * **Comment:** This field represents a boolean data indicating wheather or not the flight delayed. We have the same, and even more detailed information, in the fields with the delay in minutes
34.	**DepartureDelayGroups  45. ArrivalDelayGroups**
    * **Description:** Departure/Arrival Delay intervals, every (15 minutes from <-15 to >180)
    * **Data type:** category
    * **Keep**
    * **Comment:** This categorical data can be useful for prediction model instead of actual delay time. We have to decide and choose it later.
35.	**DepTimeBlk  46. ArrTimeBlk**
    * **Description:** CRS Departure Time Block, Hourly Intervals
    * **Data type:** category
    * **Keep**
    * **Comment:** This categorical data probably will be more usefull for the prediction model comparing to the acrual departure or arrival times in minutes. 

The folllowing data represent some time and duration for processes which I think highly correlated with data described above. I suppose this data (below) doesn't add any value to the prediction model, and I am going to DROP it.  
36.	TaxiOut: Taxi Out Time, in Minutes  
37.	WheelsOff: Wheels Off Time (local time: hhmm)  
38.	WheelsOn: Wheels On Time (local time: hhmm)  
39.	TaxiIn: Taxi In Time, in Minutes  



Let's check the fields supposed to be kept for null values and incosistent data 

In [ ]:
# Check for values of the Departure block fields
flights_2014_1.loc[:, DepFields].describe()

In [ ]:
# Check for values of the Arrival block fields
flights_2014_1.loc[:, ArrFields].describe()

In [ ]:
# Check for NA-values in the Departure block fields
flights_2014_1.loc[:, DepFields].isna().sum()

In [ ]:
# Check for NA-values in the Arrival block fields
flights_2014_1.loc[:, ArrFields].isna().sum()

In [ ]:
print(flights_2014_1['DepartureDelayGroups'].unique())
print(flights_2014_1['ArrivalDelayGroups'].unique())

In [ ]:
print(sorted(flights_2014_1['DepTimeBlk'].unique()))
print(sorted(flights_2014_1['ArrTimeBlk'].unique()))

#### Problems to analyse and solve:
1. **DepTime** and **ArrTime** contain time values '2400' (at the same time CRS times contain only 2359). I have to convert it into 0h00m of the next day.
2. It is needed to convert time into simple number of minutes from the start of the day.
3. DepTime has the same number of NA-values as DepDelay, but ArrTime and ArrDelay have number of NA-values different of those from NA-values in DepTime and DepDelay. Reason can be related to cancelled and diverted flights. There is need to check.
4. Delay Groups (Arrival and Departure) contain NaN values. Need to investigate.

In [ ]:
# Test that all flight wtth NA Departure Time (DepTime) were cancelled
print('The Cancelled field has only these values:', flights_2014_1['Cancelled'].unique())
print('In this dataset there are', int(flights_2014_1['Cancelled'].sum()), 'cancelled flights in total')
print('Among', flights_2014_1.DepTime.isna().sum(), 'flights with NA DepTime there are', 
      flights_2014_1[flights_2014_1.DepTime.isna()]['Cancelled'].count(),'cancelled flights')

Ok. We see that all NA-values in DepTime field are explained by the flight cancellaiton. 
However, there is one more interesting thing: the total number of cancelled flight is higher than the number of NA-values in DepTime field. Does it mean that these flight were cancelled but still have departure time? Let's take a look at this. 

In [ ]:
# How many Cancelled and Diverted flight are there in total?
flights_2014_1[['Cancelled', 'Diverted']].agg('sum')

In [ ]:
# What is a split of flights with/NA Departure Time vs. Cancelled and Diverted flights 
flights_2014_1.groupby(flights_2014_1['DepTime'].isna())[['Cancelled', 'Diverted']].agg('sum')

In [ ]:
# Check all cancelled flight with existing departure time don't have Time in Air (flight time)
departured_cancelled_flights = (~flights_2014_1['DepTime'].isna()) & (flights_2014_1['Cancelled'] == 1)
print('Are all flights depatured but cancelled times have NA as AirTime?',
      flights_2014_1[departured_cancelled_flights]['AirTime'].isna().all())

In [ ]:
# Investigating which flights have NA DepartureDelayGroups
NA_DepDelay_group = flights_2014_1['DepartureDelayGroups'].isna()
print('Number of NA_values in DepartureDelayGoups:', NA_DepDelay_group.sum())

cancelled_before_depurture = flights_2014_1['DepTime'].isna() & (flights_2014_1['Cancelled'] == 1)
print('Are all cancelled prior departure flights have NA-value in DepartureDelayGroups?',
      flights_2014_1[cancelled_before_depurture]['DepartureDelayGroups'].isna().all())

1. Departured flights still could be cancelled after departure (didn't take off, ruturned to gate) OR diverted to a different destination.
2. All flight with NA actual Departure Time ('DepTime' field) were canceled and have NA-Value in DepartureDelayGroups

Let’s examine the Arrival times in more detail.

In [ ]:
# What is a split of flights with/NA Arrival Time vs. Cancelled and Diverted flights 
flights_2014_1.groupby(flights_2014_1['ArrTime'].isna())[['Cancelled', 'Diverted']].agg('sum')

From this split it is interesting that some diverted (directed to other airports) flights still have Arrival time. What does this arravel time mean? Is it the arrival time to the Destination airport or the airport where the flight had beed diverted?
Le't examine this question using the field 'DivReachedDest' marking the flights reached Destination after being diverted.

In [ ]:
# Check the numbers from diverted flights' split over w/NA ArrTime 
diverted_but_arrived = (~flights_2014_1['ArrTime'].isna()) & (flights_2014_1['Diverted'] == 1)
print('Number of diverted flights that have ArrTime', 
      flights_2014_1[diverted_but_arrived]['Flight_Number_Reporting_Airline'].count())
print('Number of diverted flights reached initial destination', 
      flights_2014_1['DivReachedDest'].sum())
print('Are all these the same flitghts?',
      flights_2014_1[diverted_but_arrived]['DivReachedDest'].sum() == flights_2014_1['DivReachedDest'].sum())

So, yes, all diverted flights which finally reached their initial destination have the Arrival time.

1. All flights having an actual Arrival Time ('ArrTime') split over flied directly from Origin to Destination OR were diverted but finally reached the Destination
2. All flights with NA ArrTime were wheather canceled or diverted and landed in different Destination

In [ ]:
# Investigating which flights have NA ArrivalDelayGroups
NA_ArrivalDelay_groups = flights_2014_1['ArrivalDelayGroups'].isna()
flight_cancelled_OR_diverted = (flights_2014_1['Cancelled'] == 1) | (flights_2014_1['Diverted'] == 1)
print('The number of flight with NA-value in ArrivalDelayGroups', NA_ArrivalDelay_groups.sum())
print('Are all flights with NA-value ArrivalDelayGroups were canceled or diverted?',
      flights_2014_1[NA_ArrivalDelay_groups & flight_cancelled_OR_diverted]['Flight_Number_Reporting_Airline'].count() ==
      flights_2014_1[NA_ArrivalDelay_groups]['Flight_Number_Reporting_Airline'].count())

All Diverted or Cancelled flights have NA-value ArrivalDelayGroup

### Flight Status and Reasons for Delay  

I’m going to keep the following fields because they contain information that can be useful for interpreting the departure and arrival time fields and for calculating the actual elapsed time, respectively:

47.	**Cancelled**
    * **Description:** Cancelled Flight Indicator (1=Yes)  
    * **Data type:** boolean
    * **Keep**

48.	**CancellationCode**
    * **Description:** Specifies The Reason For Cancellation
    * **Data type:**  category
    * **Keep**

49.	**Diverted**
    * **Description:** Diverted Flight Indicator (1=Yes)  
    * **Data type:** boolean
    * **Keep**
    

Next fileds I suppose to keep to analyse the correclation between these reasons for delay with specific airlines, airport or states:

56.	**CarrierDelay**
    * **Description**: Carrier Delay, in Minutes  
    * **Data type:** float
    * **Keep**

57.	**WeatherDelay**  
    * **Description**: Weather Delay, in Minutes  
    * **Data type:** float
    * **Keep**

58.	**NASDelay** 
    * **Description**: National Air System Delay, in Minutes  
    * **Data type:** float
    * **Keep**

59.	**SecurityDelay**  
    * **Description**: Security Delay, in Minutes  
    * **Data type:** float
    * **Keep**

60.	**LateAircraftDelay**
    * **Description**: Late Aircraft Delay, in Minutes  
    * **Data type:** float
    * **Keep**

In [ ]:
# Discovering the values and NA-values
flight_status_fields = ['Cancelled',
                        'CancellationCode',
                        'Diverted']
print(flights_2014_1[flight_status_fields].describe())
print(flights_2014_1[flight_status_fields].isna().sum())

In [ ]:
# Which values does the Cancellatino Code field have?
flights_2014_1['CancellationCode'].unique()

In [ ]:
# Examining the presence of Cancellation Codes for all Cancelled flights
print(flights_2014_1.groupby(['Cancelled', 'Diverted'])['CancellationCode'].agg('count'))
no_cancellation_code = flights_2014_1['CancellationCode'].isna()
print('Number of recortds (flights) with absent Cancellation Code but still were cancelled:',
      len(flights_2014_1[no_cancellation_code & ('Cancelled' == 0)]))

The conclusion is that the data represented in the Flight Status and Cancellation Code fields is accurate and comprehensive.

### Elapsed time  

The scheduled and actual Elapsed Time data are the primary candidates for the outcome of the proposed prediction model, or at least one of the main components for constructing such a variable. This is why it is important to keep this data. Air Time can also be useful because, as we have already seen, it helps with identifying flights that were cancelled after departure.

50.	***CRSElapsedTime**
    * **Description:** CRS Elapsed Time of Flight, in Minutes  
    * **Data type:** float
    * **Keep**

51.	**ActualElapsedTime**  
    * **Description:** Elapsed Time of Flight, in Minutes  
    * **Data type:** float
    * **Keep**

52.	**AirTime:**  
    * **Description:** Flight Time, in Minutes  
    * **Data type:** float
    * **Keep**


The following data can be dropped because it is seemed irrelevand or can be obtaing from another fields:  

53.	**Flights**  
    * **Description:** Number of Flights  
    * **Drop**  

54.	**Distance**  
    * **Description:** Distance between airports (miles)  
    * **Drop**  

55.	**DistanceGroup**
    * **Description:**  Distance Intervals, every 250 Miles, for Flight Segment  
    * **Drop**
 
61.	**FirstDepTime**  
    * **Description:** First Gate Departure Time at Origin Airport  
    * **Drop**

62.	**TotalAddGTime**  
    * **Description:** Total Ground Time Away from Gate for Gate Return or Cancelled Flight  
    * **Drop**

63.	**LongestAddGTime**  
    * **Description:** Longest Time Away from Gate for Gate Return or Cancelled Flight  
    * **Drop**

64.	**DivAirportLandings**
    * **Description:** Number of Diverted Airport Landings  
    * **Drop**

As we already know, the next fields can be useful in evaluation of an acrual elapsed time when the flight was diverted, so we need to keep them:  

65.	**DivReachedDest**  
    * **Description:** Diverted Flight Reaching Scheduled Destination Indicator (1=Yes)  
    * **Data type:** boolean
    * **Keep**

66.	**DivActualElapsedTime**
    * **Description:** Elapsed Time of Diverted Flight Reaching Scheduled Destination, in Minutes. The ActualElapsedTime column remains NULL for all diverted flights.  
    * **Data type:** float
    * **Keep**

67.	**DivArrDelay**  
    * **Description:** Difference in minutes between scheduled and actual arrival time for a diverted flight reaching scheduled destination. The ArrDelay column remains NULL for all diverted flights.  
    * **Data Type:** float
    * **Keep**

The distance beetween the scheduled destination and final diverted airport in miles is unimportant for the purposed model because if the flight landed in different location, it is reasonable for the purpose of model consider this fliaght as not arrived to the distanation.

68.	**DivDistance**
    * **Description:** Distance between scheduled destination and final diverted airport (miles). Value will be 0 for diverted flight reaching scheduled destination.  
    * **Drop**

In [ ]:
# Checking for possible and NA values for Elapsed time data block
elapsed_time_fields = ['CRSElapsedTime',
                       'ActualElapsedTime',
                       'AirTime']
print(flights_2014_1[elapsed_time_fields].describe())
print(flights_2014_1[elapsed_time_fields].isna().sum())

In [ ]:
# Checking for possible and NA values for Diverted flights data block
diverted_flights_fields = ['DivReachedDest',
                       'DivActualElapsedTime',
                       'DivArrDelay']
print(flights_2014_1[diverted_flights_fields].describe())
print(flights_2014_1[diverted_flights_fields].isna().sum())

### Another infermation about diverted flights

The rest of the dataset contains 5 equal blocks for five airports where the flight can be diverted consiquently. Each block contains:
* Diverted Airport Code, Airport ID of Diverted Airport,  
* Airport Sequence ID of Diverted Airport,  
* Wheels On Time (local time: hhmm) at Diverted Airport Code,  
* Total Ground Time Away from Gate at Diverted Airport Code,  
* Longest Ground Time Away from Gate at Diverted Airport Code,  
* Wheels Off Time (local time: hhmm) at Diverted Airport Code,  
* Aircraft Tail Number for Diverted Airport Code

All this information is not rellevant to the project. 

### Conclustion about data structure and quality:  

1. The data needed for the project is selected
2. The proposed data type for each field is selected
3. The quality of the data is good. The data represented in the dataset is mostly comprehesive, but I will examine it in more detail in next section.
4. The logic of data for calncelled and diverted flights and consiquencies for actual departure and arrival times, time delays and elapsed times was identified and recorded
5. On the data transformaiton stage I have to change data types for selected for the model fields and solve the 'time-2400' problem.
6. The calculation of the resulting elapsed time, which I suppose to take as a predicted variable of the model, will be realise further on the stage of features engeneering. 

## Evaluating the Accuracy of Scheduled and Actual Elapsed Times

The elapsed times (scheduled and actual), along with departure and arrival times, are expected to play a crucial role in the model. Therefore, I must give special attention to their quality and accuracy.

There are 60 data files, which makes it difficult to check all the data at once, so I plan to start by examining one file, ‘report_2014_3.csv,’ to design an approach for handling potential errors. In the future, I will apply this approach to the remaining files.

I chose March because the start of Daylight Saving Time (DST) occurs in this month, which may introduce errors into our data.

In [2]:
# These are fields I plan to examine
data_types = {
    'FlightDate':           'str',
    'Flight_Number_Reporting_Airline':  np.int16,
    'Origin':               'category',
    'Dest':                 'category',
    'CRSDepTime':           np.int16,
    'DepTime':              np.float32,
    'DepDelay':             np.float32,
    'CRSArrTime':           np.int16,
    'ArrTime':              np.float32,
    'ArrDelay':             np.float32,
    'Cancelled':            np.int8,        # boolean
    'Diverted':             np.int8,        # boolean
    'CRSElapsedTime':       np.float32,
    'ActualElapsedTime':    np.float32,
    'AirTime':              np.float32}

In [9]:
def load_data_from(zip_file, data_file, field_type=None):
    '''
    Description
    -----------
    Load data specified as field_type from one data file from zip-archive 

    Parameters
    -----------
    zip_file - path and name of source zip-file contaning 60 csv files
    dat_faile - path and name of csv-file with data
    field_type - dictinary with fields to load and thiers relative data types

    Returns
    ==========
    DataFrame with data loaded
    '''

    # reading the file
    with zipfile.ZipFile(zip_file) as zip_source:
        with zip_source.open(data_file) as file:
            if field_type != None:
                df = pd.read_csv(file, header = 0, 
                                usecols = field_type.keys(),
                                dtype = field_type)
            else:
                df = pd.read_csv(file, header = 0, low_memory=False)

    # Converting dates and boolean        
    if 'FlightDate' in df.columns:
        df['FlightDate'] = pd.to_datetime(df['FlightDate'])
    if 'DivReachedDest' in df.columns:
        df['DivReachedDest'] = df['DivReachedDest'].fillna(0)
    if 'Cancelled' in df.columns:
        df['Cancelled'] = df['Cancelled'].astype('bool')
    if 'Diverted' in df.columns:
        df['Diverted'] = df['Diverted'].astype('bool')
    if 'DivReachedDest' in df.columns:
        df['DivReachedDest'] = df['DivReachedDest'].astype('bool')
    return df

In [46]:
# The most important combinations of fields for analysis
Date_details = ['FlightDate']
DepTime_details = ['CRSDepTime', 'DepTime']
ArrTime_details = ['CRSArrTime', 'ArrTime']
DepTime_min_details = ['CRSDepTime_min', 'DepTime_min']
ArrTime_min_details = ['CRSArrTime_min', 'ArrTime_min']
ElapsedTime_details = ['CRSElapsedTime', 'ActualElapsedTime']
CRS_details = ['CRSDepTime', 'CRSArrTime', 'CRSElapsedTime']
CRS_min_details = ['CRSDepTime_min', 'CRSArrTime_min', 'CRSElapsedTime']
Route_datails = ['Origin', 'Dest']

In [47]:
# Loading some dataset just for tests
source_zip = 'data/interim/csv_flight.zip'
source_path = 'csv_flight/report_'

flights = load_data_from(source_zip, source_path + '2014_3.csv', data_types)

In [ ]:
flights.info()

To evaluate the accuracy of departure times, arrival times, and durations (both scheduled and actual), I need to calculate the UTC datetimes for both departure and arrival. The original data includes the scheduled flight date, along with two sets of data (scheduled and actual) for departure and arrival times, as well as elapsed time. To calculate the scheduled departure datetime in the local timezone, I can combine the ‘FlightDate’ with the departure time from ‘CRSDepTime.’ For the arrival time, I first need to determine the correct date, as flights may arrive on the next day — or even the previous day — due to timezone differences. I plan to use a criterion based on the difference between the departure and arrival times to indicate a change in date. A threshold for this time difference could serve as the indicator.

In original data set all time data is provided in format HHMM, which is not conviniet for the purposes of calculation and comparison. I convert it into the simple number of minutes from midnight. 

In [2]:
# [TRANSFORMATION]
# # Convert Arrival and Departure times (CRS and Actual) to minutes
def time_to_minutes(df):
    return df // 100 * 60 + df % 100

In [50]:
flights['CRSArrTime_min'] = time_to_minutes(flights['CRSArrTime'])
flights['CRSDepTime_min'] = time_to_minutes(flights['CRSDepTime'])
flights['ArrTime_min'] = time_to_minutes(flights['ArrTime'])
flights['DepTime_min'] = time_to_minutes(flights['DepTime'])

My hypothesis is that all flights arriving the next day will have a significant negative difference between arrival and departure times (in minutes). For example, if a flight departs at 1420 min (equivalent to 23:40) and arrives at 75 minutes  (equivalent to 1:15 AM), it definitely arrived the next day. However, the negative differece can occur not only when the date changes but also when a flight departs from one timezone and arrives in another where the time difference with UTC is larger. In this case, the flight’s duration should be less than the time difference. Let’s examine this for short flights (less than 60 minutes) to chose a correct threshhold. 

In [ ]:
# Filter for flights when CRS Arrival time is less of equal than CRS Departure time
CRSArr_less_than_CRSDep_min = flights['CRSArrTime_min'] <= flights['CRSDepTime_min']
less_than_60_min = flights['CRSElapsedTime'] < 60

# FOR TIMES EXPRESSED IN MINUTES !!!
# Flights where CRS Arrival time less than CRS Departure time AND a flight was less than 60 min
# To evaluate is it possible to 'travel in time' - to arrive the same day earlier 
flights[CRSArr_less_than_CRSDep_min & less_than_60_min][Date_details + CRS_min_details] \
    .sort_values('CRSElapsedTime').head()

We see in the table above the first three flight have negative difference and definitely arrived the next day, but the last two also have negative difference but arrived the same day.

In [ ]:
# ... and the END of the table
flights[CRSArr_less_than_CRSDep_min & less_than_60_min][Date_details + CRS_min_details] \
    .sort_values('CRSElapsedTime').tail()

In this table all flights have negative difference between arrival and departure time but all of them arrived the same day

In [53]:
# [TRANSFORMATION]
# Calculation of the difference between departure and arrival time 
flights['CRS_Arr_minus_Dep'] = flights['CRSArrTime_min'] - flights['CRSDepTime_min'] 

In [ ]:
# Examine the difference between CRS Arrival and Departure times
bins = np.arange(-1440, 1440, 100)
plt.hist(flights['CRS_Arr_minus_Dep'], density=True,  bins = bins)
plt.show()

We can see that the time difference between arrival and departure can be negative or possitive but it seems that there is a certain gap in negative values. Let's zoom this histogram in. 

In [ ]:
# Examine MORE CLOSELY the difference between CRS Arrival and Departure times 
bins = np.arange(-540, 60, 60)
plt.hist(flights['CRS_Arr_minus_Dep'], density=True,  bins = bins)
plt.xticks(bins)
plt.show()

We see that there is no negative values between -8 hours (-480 min) and -1 hour (-60 min). However, to be sure let's examine the data.

In [ ]:
# Are there any values between -900 and -200
diff_between_900_200 = (flights['CRS_Arr_minus_Dep'] > -480) & (flights['CRS_Arr_minus_Dep'] < -60)
print(flights[diff_between_900_200])

del(diff_between_900_200)

Yes, there is no values between -480 and -60 minutes. 
It is possible to ‘travel in time’ only for a maximum of 60 minutes. All other differences occur when flights take off before midnight and land the next day. I can use a threshold of -60 minutes to adjust the departure date by adding one day in order to calculate the arrival date.

In [ ]:
# Are there any values between -900 and -200
diff_greater_than_1300 = (flights['CRS_Arr_minus_Dep'] > 1300)
print(flights[diff_greater_than_1300])

del(diff_greater_than_1300)

There are no records with positive difference more than 1300 minutes, but when this occures it means the flight has arrived at the previouse day due to timezone difference. I will use this calculating arrival dates. 

### Calculating dates of departure and arrivel timezone naive 

#### Scheduled dates

In [3]:
def get_CRS_datetime(dates, dep_m, arr_m):
    """
    Description
    ------------
    Calculate scheduled(CRS) Departure datetime (timezone naive) based on Flight Date and Departure time (minutes)
    Calculate scheduled(CRS) Arrival datetiem (timezone naive) based on Flight Date and Arrival time (minutes). 
        Arrival time is increased by one day from Fllght Date if the difference between CRS Arrival and Departure time 
        is equal or less than -60 minutes, and decreased by one day if the difference is at least +1380 minutes.

    Parameters:
    ------------
    dates - pandas Series with FlightDate
    dep_m - pandas Seiries with CRS Departure time in minutes 
    arr_m - pandas Seiries with CRS Arrival time in minutes

    Returns:
    ------------
    departure_datetime - CRS Departure datetime (timezone naive) (pd.Series)
    arrival_datetime - CRS Arrival datetime (timezone naive) (pd.Series)
    """

    df = pd.DataFrame() 
    df['departure_datetime'] = pd.to_datetime(dates) + pd.to_timedelta(dep_m, unit='min')

    # Filter for scheduled flights arriving next or privios day
    arr_next_day_filter = arr_m - dep_m  <= -60
    arr_previous_day_filter = arr_m - dep_m  >= 1380

    # For all scheduled flights by default arrival date is eaueal to departure date
    df['arrival_datetime'] = dates

    # Adding one day to the date if flight arrived next day
    df.loc[arr_next_day_filter, 'arrival_datetime'] = df.loc[arr_next_day_filter, 'arrival_datetime'] \
                                                        + datetime.timedelta(1)
    # Adding one day to the date if flight arrived day befor departure day
    df.loc[arr_previous_day_filter, 'arrival_datetime'] = df.loc[arr_previous_day_filter, 'arrival_datetime'] \
                                                        + datetime.timedelta(-1)
    # Finally adding actual arrival time to get actual arrival datetime
    df['arrival_datetime'] = df['arrival_datetime'] + pd.to_timedelta(arr_m, 'm')

    return df['departure_datetime'], df['arrival_datetime']


In [59]:
# converting CRS times (departure and arrival into datetime with correct arrival date - the previous, the same or the next day)
flights['CRSDepTime'], flights['CRSArrTime'] = get_CRS_datetime(flights['FlightDate'], 
                                                                              flights['CRSDepTime_min'], 
                                                                              flights['CRSArrTime_min'])

In [ ]:

# Check for actual departure and arrival datetimes 
pd.set_option('display.width', 200)

CRS_Arrived_next_day = flights['CRSArrTime_min'] - flights['CRSDepTime_min']  <= -60
CRS_Arrived_previous_day = flights['CRSArrTime_min'] - flights['CRSDepTime_min']  >= 1380
print('Flights arrived next day after departure\n', 
      flights[CRS_Arrived_next_day]
      [['FlightDate', 'CRSDepTime', 'CRSDepTime', 'DepDelay', 'CRSArrTime', 'CRSArrTime']].head())
print('Flights arrived the same day as departured\n',
      flights[~CRS_Arrived_next_day]
      [['FlightDate', 'CRSDepTime', 'CRSDepTime', 'DepDelay', 'CRSArrTime', 'CRSArrTime']].head())
print('Flights arrived the previous day as departured\n',
      flights[CRS_Arrived_previous_day]
      [['FlightDate', 'CRSDepTime', 'CRSDepTime', 'DepDelay', 'CRSArrTime', 'CRSArrTime']].head())

#### Actual dates

Calculating actual date we have to take into account that even the departure date can differ from the dataset's flight date, because this date is for scheduled time, but actual time can move to the next or previous day due to flight positive or negative delay.

In [6]:
fields_to_load = {
    # 'Year':                 np.int16,               
    # 'Quarter':              np.int8,                
    # 'Month':                np.int8,                
    # 'DayofMonth':           np.int8,
    # 'DayOfWeek':            np.int8,
    'FlightDate':           'str',
    'Reporting_Airline':    'category',
    'Flight_Number_Reporting_Airline':  np.int16,
    'OriginAirportID':      'category',
    'Origin':               'category',
    'DestAirportID':        'category',
    'Dest':                 'category',
    'CRSDepTime':           np.int16,
    'DepTime':              np.float32,
    'DepDelay':             np.float32,
    'DepartureDelayGroups': 'category',
    'DepTimeBlk':           'category',
    'CRSArrTime':           np.int16,
    'ArrTime':              np.float32,
    'ArrDelay':             np.float32,
    'ArrivalDelayGroups':   'category',
    'ArrTimeBlk':           'category',
    'Cancelled':            np.int8,        # boolean
    'CancellationCode':     'category',
    'Diverted':             np.int8,        # boolean
    'CarrierDelay':         np.float32,
    'WeatherDelay':         np.float32,
    'NASDelay':             np.float32,
    'SecurityDelay':        np.float32,
    'LateAircraftDelay':    np.float32,
    'CRSElapsedTime':       np.float32,
    'ActualElapsedTime':    np.float32,
    'AirTime':              np.float32,
    'DivReachedDest':       np.float32,        # boolean
    'DivActualElapsedTime': np.float32,
    'DivArrDelay':          np.float32}

In [4]:
def get_Actual_datetime(dates, dep_m, arr_m, dep_delay):
    """
    Description
    ------------
    Calculate Actual Departure datetime (timezone naive) based on Flight Date, Departure time (minutes) and Departure 
        delay (minutes)
    Calculate Actual Arrival datetiem (timezone naive) based on Flight Date, Actual Departure date in accordance with 
        Departure delay and Arrival time (minutes). Arrival time is increased by one day from Departure Date if 
        the difference between CRS Arrival and Departure time is equal or less than -60 minutes, and decreased by one day 
        if the difference is at least +1380 minutes.

    Parameters:
    ------------
    dates - FlightDate (pd.Series)
    dep_m - Actual Departure time in minutes (pd.Series)
    arr_m - Actual Arrival time in minutes (pd.Series)
    dep_delay - departure delay in minutes (pd.Series)

    Returns:
    ------------
    departure_datetime - Actual Departure datetime (timezone naive) (pd.Series)
    arrival_datetime - Actual Arrival datetime (timezone naive) (pd.Series)
    """

    # Filters for flights having DepTime_min and ArrTime_min 
    # These filters actually for not cancelled flights, but they differ each other because some flights cancelles after departure
    # So they have departure time but didn't fly 
    Dep_exists_filter = ~dep_m.isna()
    Arr_exists_filter = ~arr_m.isna()

    # Array with 'day shift' due to flight delay. NOTICE: some flights have -1 day shift because day had a small negative delay 
    # having a scheduled departure time several minutes after midnight

    d = np.zeros([len(dates)], dtype='int')
    d = ((dep_m.fillna(0) + dep_delay.fillna(0)) // 1440).astype(int)
    day_deltas_due_to_delay = pd.to_timedelta(d, unit='days')

    # Calculating actual departure datetime
    df = pd.DataFrame() #pd.NaT, index = [*range(0, len(dates))], columns= ['departure_datetime', 'arrival_datetime'])
    df.loc[Dep_exists_filter, 'departure_datetime'] = dates[Dep_exists_filter] \
                                                + day_deltas_due_to_delay[Dep_exists_filter] \
                                                + pd.to_timedelta(dep_m[Dep_exists_filter], 'm')

    # Calculating actual arrival datetime

    # Filter for flights arrived next or previous day
    Arrived_next_day = arr_m - dep_m  <= -60
    Arrived_previous_day = arr_m - dep_m  >= 1380

    # For all arrived flights at first arrival date is eaueal to departure date
    df.loc[Arr_exists_filter, 'arrival_datetime'] = dates[Arr_exists_filter] \
                                                + day_deltas_due_to_delay[Arr_exists_filter] 
    # Adding one day to the date if flight arrived next day
    df.loc[Arr_exists_filter & Arrived_next_day, 'arrival_datetime'] = \
            df.loc[Arr_exists_filter & Arrived_next_day, 'arrival_datetime'] + datetime.timedelta(1)
    
    # Adding one day to the date if flight arrived previous day
    df.loc[Arr_exists_filter & Arrived_previous_day, 'arrival_datetime'] = \
            df.loc[Arr_exists_filter & Arrived_previous_day, 'arrival_datetime'] + datetime.timedelta(-1)
    
    # Finally adding actual arrival time to get actual arrival datetime
    df.loc[Arr_exists_filter, 'arrival_datetime'] = \
            df.loc[Arr_exists_filter, 'arrival_datetime'] + pd.to_timedelta(arr_m.loc[Arr_exists_filter], 'm')
    
    return df['departure_datetime'], df['arrival_datetime']

In [63]:
flights['ActDepTime'], flights['ActArrTime'] = get_Actual_datetime(flights['FlightDate'], 
                                                                                  flights['DepTime_min'], 
                                                                                  flights['ArrTime_min'],
                                                                                  flights['DepDelay'])

In [ ]:
# Check for actual departure and arrival datetimes 
Arrived_next_day = flights['ArrTime_min'] - flights['DepTime_min']  <= -60
Arrived_previous_day = flights['ArrTime_min'] - flights['DepTime_min']  >= 1380
check_fields = ['FlightDate', 'DepTime', 'ActDepTime', 'DepDelay', 'ActArrTime', 'ArrTime']

print('Flights arrived next day after departure\n', flights[Arrived_next_day][check_fields].head())
print('Flights arrived the same day as departured\n', flights[~Arrived_next_day][check_fields].head())
print('Flights arrived the previous day as departured\n', flights[Arrived_previous_day][check_fields].head())

#### Adding local timezone derived from IATA code of airport

In [7]:
# [TRANSFORMATION]
# Function to add to the series of datetime a timezone specified by airports IATA codes
def add_local_tz(df, dt_field, IATA_code_field):
    return [row[dt_field].tz_localize(tz=tz.gettz(IATAtz[row[IATA_code_field]]), ambiguous=True ,nonexistent='shift_forward')
        for _, row in df.iterrows()]

In [53]:
'''
# [TRANSFORMATION]
# Adding local timezones to the Departure and Arrival times 
flights['CRSDepTime'] = add_local_tz(flights, 'CRSDepTime', 'Origin')
flights['CRSArrTime'] = add_local_tz(flights, 'CRSArrTime', 'Dest')
flights['ActDepTime'] = add_local_tz(flights, 'ActDepTime', 'Origin')
flights['ActArrTime'] = add_local_tz(flights, 'ActArrTime', 'Dest')
'''

In [ ]:
# check CRS fields
check_fields_CRS = ['FlightDate', 'CRSDepTime', 'CRSDepTime', 'CRSArrTime', 'CRSArrTime', 'CRSElapsedTime']
print(flights[CRSArr_less_than_CRSDep_min][check_fields_CRS].head())
print(flights[~CRSArr_less_than_CRSDep_min][check_fields_CRS].head())

In [ ]:
# check ACTUAL fields
check_fields_CRS = ['FlightDate', 'ActDepTime', 'DepTime', 'ActArrTime', 'ArrTime', 'ActualElapsedTime']
print(flights[Arrived_next_day][check_fields_CRS].head())
print(flights[~Arrived_next_day][check_fields_CRS].head())

Ok, it seems that our dates are properly localized to the respective timezone. 

#### Converting datetimes to UTC timezone

In [5]:
# Function to convert datetime timezone naive to UTC timezone
def convert_to_UTC(df, dt_field, IATA_code_field):
    return [row[dt_field].tz_localize(tz=tz.gettz(IATAtz[row[IATA_code_field]]), ambiguous=True, nonexistent='shift_forward')
             .astimezone(tz.UTC) for _, row in df.iterrows()]

In [70]:
# Getting UTC timezone for scheduled (CRS) departure and arrival times
flights['CRSDep_UTC'] = convert_to_UTC(flights, 'CRSDepTime', 'Origin')
flights['CRSArr_UTC'] = convert_to_UTC(flights, 'CRSArrTime', 'Dest')

# We have to filter for non-NA values because astimezone doesn't work wiht NaN/NaT
nonNA_Dep = ~flights['ActDepTime'].isna()
flights.loc[nonNA_Dep, 'ActDep_UTC'] = convert_to_UTC(flights[nonNA_Dep], 'ActDepTime', 'Origin')

# We have to filter for non-NA values because astimezone doesn't work wiht NaN/NaT
nonNA_Arr = ~flights['ActArrTime'].isna()
flights.loc[nonNA_Arr, 'ActArr_UTC'] = convert_to_UTC(flights[nonNA_Arr], 'ActArrTime', 'Dest')

In [ ]:
# Check CRS
fields_to_check_UTC_CRS = ['CRSDepTime', 'CRSDep_UTC', 'CRSArrTime', 'CRSArr_UTC']
print(flights[fields_to_check_UTC_CRS].head())

# Check ACTUAL
fields_to_check_UTC_Act = ['ActDepTime', 'ActDep_UTC', 'ActArrTime', 'ActArr_UTC']
print(flights[fields_to_check_UTC_Act].head())

#### Calculating scheduled and actual elapsed times based on UTC departure and arrival

In [72]:
# [TRANSFORTATION]
# Calculating Elapsed time based on UTC times
flights['UTCElapsedTime_CRS'] = (flights['CRSArr_UTC'] - flights['CRSDep_UTC']).dt.total_seconds() / 60

flights.loc[nonNA_Arr, 'UTCElapsedTime_Act'] = pd.to_timedelta((flights.loc[nonNA_Arr, 'ActArr_UTC'] - 
                                                                flights.loc[nonNA_Arr, 'ActDep_UTC'])).dt.total_seconds() / 60



In [73]:
# [TRANSFORMATINO]
# The difference between UTC-based Elapsed time and CRS/ACTUAL Elapsed time
flights['diff_CRS'] = flights['UTCElapsedTime_CRS'] - flights['CRSElapsedTime']
flights['diff_Act'] = flights['UTCElapsedTime_Act'] - flights['ActualElapsedTime']

### Examining the accuracy of CRS Elapsed time

In [ ]:
# Examine the flights having difference between UTC-Based and CRS Elapsed times 
diff_not_zero = flights[flights['diff_CRS'] != 0][fields_to_check_UTC_CRS + ['CRSElapsedTime', 'UTCElapsedTime_CRS']]
print('There are', len(diff_not_zero), 'flights where CRS Elapsed time differ from CRS Elapsed time culculated based on UTC')
diff_not_zero.head()

del(diff_not_zero)

In [ ]:
flights['diff_CRS'].value_counts()

There are 76 flights in the datased where we have a difference between UTC and CRS elapsed time.


Let’s take a detailed look at one of these flights: the first one from the table above, with ID = 86022, which has a -60 minute difference between UTC and CRS elapsed times.

In [ ]:
# Let's take a look on the first flight with diffeence
flights.iloc[86022]

First, we see that this flight took place during the spring 2014 DST, and the difference between UTC and CRS elapsed times is exactly 60 minutes. But which time is correct? Let’s take a look at this flight on other days.

In [ ]:
flights[(flights['Flight_Number_Reporting_Airline'] == 103) &
         (flights['Origin'] == 'SEA')][['CRSArrTime', 'CRSElapsedTime', 'UTCElapsedTime_CRS', 'diff_CRS']]

It seems that the only flight having the difference is the filight landed 9th March 2014 and the UTC-based elapsed time is correct (CRSElapsedTime has the wrong value for DST day). Let's examine other flights having difference of 60 min.

In [ ]:
diff_60_min = flights[flights['diff_CRS'] == -60][['CRSDepTime', 'CRSArrTime', 'CRSElapsedTime', 'UTCElapsedTime_CRS', 'diff_CRS']]
print(len(diff_60_min), 'records')
diff_60_min

del(diff_60_min)

We can see that all flights with a -60 minute difference between UTC and CRS elapsed times occurred during DST. There are 13 such records in the dataset.

We can conclude that the CRS elapsed time is incorrect for all 13 flights that passed through DST.

This explains the discrepancy between UTC and CRS elapsed times for 13 out of the 76 flights in the dataset. But what about the remaining flights with a similar difference?

In [ ]:
flights['UTCElapsedTime_CRS'].describe()

There is another interesting fact - some flights has also engative UTC elapsed time! How is this possible?

In [ ]:
error_less_0 = flights[flights['UTCElapsedTime_CRS'] < 0]
print(len(error_less_0))
error_less_0[
    ['CRSDepTime', 
     'Origin', 
     'CRSArrTime', 
     'Dest', 
     'CRSElapsedTime', 
     'UTCElapsedTime_CRS', 
     'diff_CRS']]


There are 31 flights with a negative UTC elapsed time. Previously, we analyzed the differences between UTC and CRS elapsed times and found that 31 flights had a -1440 minute difference. These are the same flights. A difference of 1440 minutes is equivalent to 24 hours.

All 31 flights were from HNL to GUM, which is interesting because HNL is in the UTC-10 timezone, while GUM is in the UTC+10 timezone. These two airports are located across the International Date Line (IDL).

In [ ]:
print(error_less_0[['CRSDep_UTC', 'CRSDepTime', 'Origin', 'CRSArr_UTC', 'CRSArrTime', 'Dest', 'diff_CRS']].head())

del(error_less_0)

The easiest way to correct this is to find all flight with UTC Departures less than UTC Arrival and add 24 x 60 minutes to the UTC Elapsed time

There ara other flights which have the "oposite" difference - they differ on +1440 minutes. 

In [ ]:
error_plus_1440 = flights[flights['diff_CRS'] == 1440]
print(len(error_plus_1440))
print(error_plus_1440[['CRSDep_UTC', 'CRSDepTime', 'Origin', 'CRSArr_UTC', 'CRSArrTime', 'Dest', 'diff_CRS']])

del(error_plus_1440)

All these flight are back flights from GUM to HLN across Internation Date Line (IDL). The ramedy is simillar - to subtract 24 x 60 (1440) minutes from UTC elapsed time. 

There is just one flight with unexamined yet difference. 

In [ ]:
error_minus_36 = flights[flights['diff_CRS'] == -36]
print(len(error_minus_36), 'records')
print(error_minus_36[['Flight_Number_Reporting_Airline', 'CRSDep_UTC', 'CRSDepTime', 'Origin', 'CRSArr_UTC', 
               'CRSArrTime', 'Dest', 'CRSElapsedTime', 'UTCElapsedTime_CRS', 
               'ActualElapsedTime', 'diff_CRS', 'Cancelled', 'Diverted']].T)

del(error_minus_36)

In [ ]:
flights[(flights['Flight_Number_Reporting_Airline'] == 317) &
        (flights['Origin'] == 'DEN') &
        (flights['Dest'] == 'PSP')] [['CRSElapsedTime', 'UTCElapsedTime_CRS', 'diff_CRS']]

In this case we see that CRS Elapsed time is:
1. also inconsistent with CRS Departure and CRS Arrival times
2. all other flights for this route have the equal CRS and UTC elapsed times which is 129 minutes  

So this can be an error

### Checking ACTUAL time

In [ ]:
flights['diff_Act'].value_counts()

It appears that the actual elapsed time shows the same differences as the scheduled elapsed time between the two types of flight duration: one calculated using UTC arrival and departure times, and the other based on the ActualElapsedTime provided in the dataset. Let’s take a closer look.

The total number of discrepancies is 78, with the following breakdown:
*	+/-1440 minutes: 60 records, likely related to flights crossing the International Date Line (IDL) from HNL to GUM and back.
*	-60 minutes: 15 records, possibly due to flights occurring during the transition to Daylight Saving Time (DST) in the spring.
*	Other: 3 records with small, random differences, which could be attributed to minor errors.

In [ ]:
error_1440 = flights[flights['diff_Act'].isin([1440, -1440])]
print('There are:', len(error_1440), 'records with differenc +/-1440 min (equal to 24 hours)')
print('Which airports are presented among these flights:', *set(error_1440['Origin']).union(set(error_1440['Origin'])))

del(error_1440)

All these flights are again between two airports across IDL. We already know this problem and know how to cue it. 

In [87]:
error_60 = flights[flights['diff_Act'] == -60][['Flight_Number_Reporting_Airline', 'ActDepTime', 'ActArrTime',
                                           'ActDep_UTC', 'ActArr_UTC', 'Origin', 'Dest', 'ActualElapsedTime', 'UTCElapsedTime_Act']]

In [ ]:
error_60['DST_Origin'] = pd.to_datetime("2014-03-09 01:59:59")
error_60['DST_Dest'] = pd.to_datetime("2014-03-09 02:00:01")
error_60['DST_Origin'] = add_local_tz(error_60, 'DST_Origin', 'Origin')
error_60['DST_Dest'] = add_local_tz(error_60, 'DST_Dest', 'Dest')

print((error_60['ActDepTime'] < error_60['DST_Origin']).all())
print(error_60[~(error_60['ActArrTime'] > error_60['DST_Dest'])].T)

All these flights departured before the transition to DST and arrived after it. This is the reason they have a wrong ActualElapsedTime. It's worth to mention that theirs arrival time is correct.


In [ ]:
other = flights[(flights['diff_Act'] > -60) & (flights['diff_Act'] < 10) & (flights['diff_Act'] != 0)]
print(len(other))
other[['Flight_Number_Reporting_Airline',
       'ActDepTime', 
       'ActArrTime',
       'Origin', 
       'Dest', 
       'ActDep_UTC',
       'ActArr_UTC',
       'diff_Act', 
       'ActualElapsedTime', 
       'UTCElapsedTime_Act']].T

First of all, we notice that all these differences appear again on the spring DST start date.
Secondly, it looks like the first two flights have minor errors that we can disregard, and we can use UTC elapsed time instead of Actual elapsed time. However, the third flight has a significant difference. Let’s examine it in more detail.

In [ ]:
flights[(flights['Flight_Number_Reporting_Airline'] == 5702) &\
        (flights['Origin'] == 'ORD') &\
        (flights['Dest'] == 'CAE')] [['Flight_Number_Reporting_Airline',
       'ActDepTime', 
       'ActArrTime',
       'Origin', 
       'Dest', 
       'ActDep_UTC',
       'ActArr_UTC',
       'diff_Act', 
       'ActualElapsedTime', 
       'UTCElapsedTime_Act']]

We observe that the same flight on other days has a duration close to the actual elapsed time recorded on March 9th. The elapsed time based on UTC, however, is significantly different from the real duration, which suggests that either the actual departure or arrival time (or both) is incorrect. There are several options to address this, such as adjusting the departure or arrival time based on the average duration of similar flights. However, due to the insignificance of this error (1 out of almost 500k flights in total, and 25 flights on this route in the same month), the simplest solution is to drop this record on the next stage.

### Conclusions for evaluation of the accuracy of Scheduled and Actual Elapsed Times  
1. The most of data accurate
2. The flights across IDL (particularly GUM and HNL airports) reqieres substructuo of 1440 minutes (24hours) due to the date change.
3. The start of DST creates minor errors in elapsed time calculation and requires correction.
4. Therea are absulute minorite of flights that have indiscrepencies between elapsed time and departur and arrival time (3 flight for more about 500k records for March 2014.)

## Data transformation

In [93]:
fields_to_load = {
    # 'Year':                 np.int16,               
    # 'Quarter':              np.int8,                
    # 'Month':                np.int8,                
    # 'DayofMonth':           np.int8,
    # 'DayOfWeek':            np.int8,
    'FlightDate':           'str',
    'Reporting_Airline':    'category',
    'Flight_Number_Reporting_Airline':  np.int16,
    # 'OriginAirportID':      'category',
    'Origin':               'category',
    # 'DestAirportID':        'category',
    'Dest':                 'category',
    'CRSDepTime':           np.int16,
    'DepTime':              'float32',
    'DepDelay':             'float32',
    'DepartureDelayGroups': 'category',
    'DepTimeBlk':           'category',
    'CRSArrTime':           np.int16,
    'ArrTime':              'float32',
    'ArrDelay':             'float32',
    'ArrivalDelayGroups':   'category',
    'ArrTimeBlk':           'category',
    'Cancelled':            np.int8,        # boolean
    'CancellationCode':     'category',
    'Diverted':             np.int8,        # boolean
    'CarrierDelay':         'float32',
    'WeatherDelay':         'float32',
    'NASDelay':             'float32',
    'SecurityDelay':        'float32',
    'LateAircraftDelay':    'float32',
    'CRSElapsedTime':       'float32',
    'ActualElapsedTime':    'float32',
    'AirTime':              'float32',
    'DivReachedDest':       'float32',      # boolean
    'DivActualElapsedTime': 'float32',
    'DivArrDelay':          'float32'}

In [ ]:
source_zip = 'data/interim/csv_flight.zip'
source_path = 'csv_flight/report_'
counter = 0
total_time = 0

# Fields to downcast type from 'float32' to 'float16'
downcast_fields = ['DepDelay', 'ArrDelay', 'CarrierDelay', 'WeatherDelay','NASDelay', 'SecurityDelay', 'LateAircraftDelay', 
                   'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DivActualElapsedTime', 'DivArrDelay', 'UTCElapsedTime_CRS',
                   'UTCElapsedTime_Act', 'diff_CRS', 'diff_Act']

for year in range(2014, 2019):
    for month in range(1, 13): 
        t_start = time()
        counter += 1
        source_data = source_path + str(year) + '_' + str(month) + '.csv'
        print(source_data + ' ', end='')

        flights = pd.DataFrame()
        flights = load_data_from(source_zip, source_data, fields_to_load)

        flights['CRSArrTime_min'] = time_to_minutes(flights['CRSArrTime'])
        flights['CRSDepTime_min'] = time_to_minutes(flights['CRSDepTime'])
        flights['ArrTime_min'] = time_to_minutes(flights['ArrTime'])
        flights['DepTime_min'] = time_to_minutes(flights['DepTime'])

        flights['CRSDepDT'], flights['CRSArrDT'] = get_CRS_datetime(flights['FlightDate'], 
                                                                    flights['CRSDepTime_min'], 
                                                                    flights['CRSArrTime_min'])

        flights['DepDT'], flights['ArrDT'] = get_Actual_datetime(flights['FlightDate'], 
                                                                flights['DepTime_min'], 
                                                                flights['ArrTime_min'],
                                                                flights['DepDelay'])

        # Getting UTC timezone for scheduled (CRS) departure and arrival times
        flights['CRSDep_UTC'] = convert_to_UTC(flights, 'CRSDepDT', 'Origin')
        flights['CRSArr_UTC'] = convert_to_UTC(flights, 'CRSArrDT', 'Dest')

        # We have to filter for non-NA values because astimezone doesn't work wiht NaN/NaT
        nonNA_Dep = ~flights['DepTime'].isna()
        flights['Dep_UTC'] = pd.NaT
        flights['Dep_UTC'] = flights['Dep_UTC'].dt.tz_localize(tz.UTC)
        flights.loc[nonNA_Dep, 'Dep_UTC'] = pd.to_datetime(convert_to_UTC(flights[nonNA_Dep], 'DepDT', 'Origin'))

        # We have to filter for non-NA values because astimezone doesn't work wiht NaN/NaT
        nonNA_Arr = ~flights['ArrTime'].isna()
        flights['Arr_UTC'] = pd.NaT
        flights['Arr_UTC'] = flights['Arr_UTC'].dt.tz_localize(tz.UTC)
        flights.loc[nonNA_Arr, 'Arr_UTC'] = convert_to_UTC(flights[nonNA_Arr], 'ArrDT', 'Dest')

        # Calculating Elapsed time based on UTC times
        flights['UTCElapsedTime_CRS'] = (flights['CRSArr_UTC'] - flights['CRSDep_UTC']).dt.total_seconds() / 60
        flights.loc[nonNA_Arr, 'UTCElapsedTime_Act'] = pd.to_timedelta((flights.loc[nonNA_Arr, 'Arr_UTC'] - 
                                                                flights.loc[nonNA_Arr, 'Dep_UTC'])).dt.total_seconds() / 60

        flights['diff_CRS'] = flights['UTCElapsedTime_CRS'] - flights['CRSElapsedTime']
        flights['diff_Act'] = flights['UTCElapsedTime_Act'] - flights['ActualElapsedTime']
       
        # Cleaning the dataset
        flights.drop(['FlightDate', 
                    'DepTime', 'DepTime_min', 'ArrTime', 'ArrTime_min',
                    'CRSDepTime', 'CRSDepTime_min', 'CRSArrTime', 'CRSArrTime_min'], axis = 1, inplace = True)
        flights[downcast_fields] = flights[downcast_fields].astype('float16')

        # Export data
        output_file_name = 'data/processed/processed_' + str(year) + '_' + str(month) + '.pickle'
        with open(output_file_name, 'wb') as out_file:
            pickle.dump(flights, out_file, protocol=pickle.HIGHEST_PROTOCOL)

        del(flights)

        t_end = time()
        elapsed_t = t_end - t_start
        total_time += elapsed_t
        print(round(elapsed_t, 1), 'sec, total time:', round(total_time, 1), \
            'sec, avg: ', round(total_time / counter, 1), 'sec, remaining time (estimate)', 
            round(total_time / counter * (60 - counter) /60), 'min.')


csv_flight/report_2014_1.csv 

In [103]:
flights = pd.DataFrame()
input_file_name = 'data/processed/processed_' + str(2014) + '_' + str(1) + '.pickle'
print(input_file_name)
with open(input_file_name, 'rb') as in_file:
    flights = pickle.load(in_file)
flights.info()

data/processed/processed_2014_1.pickle
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471949 entries, 0 to 471948
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype                  
---  ------                           --------------   -----                  
 0   FlightDate                       471949 non-null  datetime64[ns]         
 1   Reporting_Airline                471949 non-null  category               
 2   Flight_Number_Reporting_Airline  471949 non-null  int16                  
 3   OriginAirportID                  471949 non-null  category               
 4   Origin                           471949 non-null  category               
 5   DestAirportID                    471949 non-null  category               
 6   Dest                             471949 non-null  category               
 7   CRSDepTime                       471949 non-null  int16                  
 8   DepTime                          441622 non-null  float

In [104]:
def memory_usage_per_type(data_frame):
    '''
    Source: https://medium.com/@alielagrebi/optimize-the-pandas-dataframe-memory-consuming-for-low-environment-24aa74cf9413
    '''
    types = ['number', 'object', 'datetimetz', 'category', 'bool']
    for tp in types:
        selected_col = data_frame.select_dtypes(include=[tp])
        memory_usage_per_type_b = selected_col.memory_usage(deep=True).sum()
        memory_usage_per_type_mb = memory_usage_per_type_b / 1024**2
        print('memory usage for {} columns: {:03.3f} MB'.format(tp, memory_usage_per_type_mb))
    

def memory_usage(data_frame):
    return data_frame.memory_usage(deep=True).sum() / 1024**2


In [105]:
memory_usage_per_type(flights)
print('Memory usage by dataframe {:.2f} MB'.format(memory_usage(flights)) )

memory usage for number columns: 47.709 MB
memory usage for object columns: 0.000 MB
memory usage for datetimetz columns: 14.403 MB
memory usage for category columns: 6.367 MB
memory usage for bool columns: 1.350 MB
Memory usage by dataframe 87.83 MB


In [ ]:
flights = pd.DataFrame()
flights_list = []
for year in range(2014, 2019):
    for month in range(1, 13): 
        input_file_name = 'data/processed/processed_' + str(year) + '_' + str(month) + '_v2.pickle'
        print(input_file_name)
        with open(input_file_name, 'rb') as in_file:
            flights_list.append(pickle.load(in_file))

flights = pd.concat(flights_list, axis=0, ignore_index=True)

In [4]:
del(flights_list)

In [ ]:
print(flights.info())

In [ ]:
output_file_name = 'data/processed/processed_2014_2018.pickle'
with open(output_file_name, 'wb') as out_file:
    pickle.dump(flights, out_file, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
flights.to_csv('data/processed/processed_2014_2018.csv')

In [ ]:
flights.info()

In [ ]:
selected_col = flights['ActArrTime'].memory_usage(deep=True)
selected_col

In [ ]:
flights['ActArrTime'].head()

In [ ]:
selected_col = flights['CRSDep_UTC'].memory_usage(deep=True)
selected_col

In [12]:
del(flights)

In [ ]:
# Analysis of difference between 'ActualElapsedTime' and 'CRSElapsedTime'
flights['ElapsedTimeDiff'] = flights['ActualElapsedTime'] - flights['CRSElapsedTime']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

rng = np.random.default_rng(18)
gauss_distr = rng.normal(flights['ElapsedTimeDiff'].mean(),
                        flights['ElapsedTimeDiff'].std(),
                        size=1000000)

bins = np.arange(-50,50)

plt.hist(flights['ElapsedTimeDiff'], bins=bins, density=True)
plt.hist(gauss_distr, histtype='step', bins=bins, color='r', density=True)
plt.xlim(-50,50)
plt.show()


In [ ]:
print('Mean difference between scheduled and actual time', round(flights['ElapsedTimeDiff'].mean(), 2))
print('Standard deviation of the difference', round(flights['ElapsedTimeDiff'].std(), 2))

The diffence between acheduled and actual elapsed time is not normally distribute (despite being very close), In average the actual elapsed time less than the scheduled (-4.73 min). However, this is the duration of the flight and not a arrival time.

It is interesting to examine the most negative values of the difference between scheduled and actual elapsed time: how shorter the actual duration of flight can be against its scheduled duration?

In [ ]:
flights['ElapsedTimeDiff'].describe()

Interesting!!!! Some flights lasted 250 minutes less than they were scheduled!!! Is it possiblethat these data can be wrong?

How many flights have an elapsed time difference that is an outlier in the range?
To answer this question let's first calculate the relative difference of elapsed time, because on long distanced the absolute difference in minutes can be higher.

In [142]:
# Calculating the relative difference of elapsed time
flights['RelElapTimeDiff'] = flights['ElapsedTimeDiff'] / flights['CRSElapsedTime']


In [ ]:
print('Mean relative difference between scheduled and actual time (%)', round(flights['RelElapTimeDiff'].mean() * 100, 2))
print('Standard deviation of the relative difference (%)', round(flights['RelElapTimeDiff'].std() * 100, 2))

In [ ]:
rng = np.random.default_rng(18)
gauss_distr = rng.normal(flights['RelElapTimeDiff'].mean(),
                        flights['RelElapTimeDiff'].std(),
                        size=1000000)

bins = np.arange(-1, 1, 0.01)

plt.hist(flights['RelElapTimeDiff'], bins=bins, density=True)
plt.hist(gauss_distr, histtype='step', bins=bins, color='r', density=True)
plt.xlim(-0.5, 0.5)
plt.show()

In [ ]:

q25, q50, q75 = flights['RelElapTimeDiff'].quantile([0.25, 0.5, 0.75])
outlier_limit = q25 - (q75 - q25) * 1.5
print('All flights with actual elapsed time', round(outlier_limit * 100, 2), 'percents of scheduled elapsed time are deffinitely outliers')

In [ ]:
# Filtering flights that had actual elapsed time to short relative to scheduled 
actual_time_less_than_CRRS = flights['RelElapTimeDiff'] < outlier_limit

# Printing the TOP-10 fasters flights
SuperFastFlights = flights[actual_time_less_than_CRRS]
print(SuperFastFlights[['FlightDate',
                        'Origin', 
                        'Dest',
                        'ActualElapsedTime', 
                        'CRSElapsedTime', 
                        'ElapsedTimeDiff',
                        'RelElapTimeDiff']]
      .sort_values('RelElapTimeDiff', ascending=True).head(10))

# Printing some summary for these flights
print('In 2014-2018 there were', 
      SuperFastFlights['Flight_Number_Reporting_Airline'].count(), 
      'flight arrived earlier for more than', round(outlier_limit * 100, 2), '% (', 
      round(SuperFastFlights['Flight_Number_Reporting_Airline'].count() / len(flights) * 100, 2),
      '% of total flights)')

In [ ]:
# Details for the flight ind = 21568647 which has the most significant difference between scheduled and actual 
# elapsed times
data_details = ['FlightDate']
route_details = ['Origin', 'Dest']
DepArr_details = ['CRSDepTime', 'CRSArrTime', 'DepTime', 'ArrTime']

print(flights.iloc[28530440][data_details + route_details + DepArr_details + elapsed_time_fields])
print('Flight number:', flights.iloc[28530440]['Flight_Number_Reporting_Airline'])

It seems strange that this flight was scheduled with 302 min of elapsed time (more then 5 hours!!!), because according to Google [search](https://www.google.com/maps/dir/SBP,+Airport+Drive,+San+Luis+Obispo,+CA,+USA/San+Francisco+International+Airport+(SFO),+San+Francisco,+CA,+USA/@36.4210897,-122.8306609,8z/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x80ecf6bf3876b9f1:0xf486acd07a0f3bd2!2m2!1d-120.6420121!2d35.2375699!1m5!1m1!1s0x808f778c55555555:0xa4f25c571acded3f!2m2!1d-122.3816274!2d37.6191145!3e4?entry=ttu&g_ep=EgoyMDI0MTAwMi4xIKXMDSoASAFQAw%3D%3D) shows that these airports are quite close to each other and estimated the direct flight time as 1h 5min, which is more close to the acctual elapsed time

In [ ]:
# Details for the flight ind = 26994453 which has the second most significant difference between scheduled and actual 
# elapsed times
print(flights.iloc[21568647][data_details + route_details + DepArr_details + elapsed_time_fields])
print('Flight number:', flights.iloc[21568647]['Flight_Number_Reporting_Airline'])


The second most differing flight has the same story: Google [search](https://www.google.com/maps/dir/LaGuardia+Airport+(LGA),+East+Elmhurst,+NY,+USA/Dallas+Fort+Worth+International+Airport+(DFW),+2400+Aviation+Dr,+Dallas,+TX+75261,+United+States/@36.3681193,-96.0279203,5z/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x89c25f8983424db5:0x772fc4660e9666b3!2m2!1d-73.8742467!2d40.7766422!1m5!1m1!1s0x864c2a660d222aa7:0x73323f5e067d201c!2m2!1d-97.0335765!2d32.8990434!3e4?entry=ttu&g_ep=EgoyMDI0MTAwMi4xIKXMDSoASAFQAw%3D%3D) shows the estimated time of the direct flight 3h 40m which again closer to the actual elapsed time and far away from scheduled 431 min - more than 7h!

In [ ]:
# load source file for accessing some detailes that were deleted in the result dataset
_fl = load_data_from(source_zip, source_path + '2017_10.csv')

# tiltering the flight of interest 
ind = _fl[(_fl['Flight_Number_Reporting_Airline'] == 5028) 
    & (_fl['DayofMonth'] == 9) 
    & (_fl['Origin'] == 'SBP')
    & (_fl['Dest'] == 'SFO')].index
flight_top1 = _fl.iloc[*ind]

# Printing results 
pd.set_option('display.max_rows', 115)
print(_fl.iloc[*ind][:70])

In [ ]:
# load source file for accessing some detailes that were deleted in the result dataset
_fl = load_data_from(source_zip, source_path + '2018_7.csv')

# tiltering the flight of interest 
ind = _fl[(_fl['Flight_Number_Reporting_Airline'] == 5935) 
    & (_fl['DayofMonth'] == 25) 
    & (_fl['Origin'] == 'DFW')
    & (_fl['Dest'] == 'LGA')].index
flight_top2 = _fl.iloc[*ind]

# Printing results 
pd.set_option('display.max_rows', 115)
print(_fl.iloc[*ind][:70])

These two flight were quite ordinary flights: they were not canceled or diverted, they had just one flight (field 'Fligths'), and as we know from Google search above their actual elapsed time seems reasonable but their GRSElapsedTime doesn't. 

Could the CRS Elapsed Time have been calculated incorrectly due to timezone differences?
Let's check it

In [ ]:
# The fastest flight - TOP1
def time_chech(list_of_flights):
    for a in list_of_flights:
        CRSDepTime = datetime(a.Year, a.Month, a.DayofMonth, a.CRSDepTime // 100, a.CRSDepTime % 100)
        CRSArrTime = datetime(a.Year, a.Month, a.DayofMonth, a.CRSArrTime // 100, a.CRSArrTime % 100)

        print('Flight #', a.Flight_Number_Reporting_Airline)
        print('CRS: ')
        print(a.Origin, '\nTime zone: ', tz.gettz(IATAtz[a.Origin]))
        print('Local departure time:      ', CRSDepTime)
        print('Local departure time (tz): ', airport_time_tz(CRSDepTime, a.Origin))
        print('UTC Departure time:        ', airport_time_UTC(CRSDepTime, a.Origin))
        print('')
        print(a.Dest, '\nTime zone: ', tz.gettz(IATAtz[a.Dest]))
        print('Local departure time:      ', CRSArrTime)
        print('Local departure time (tz): ', airport_time_tz(CRSArrTime, a.Dest))
        print('UTC Departure time:        ', airport_time_UTC(CRSArrTime, a.Dest))
        print('')
        print('Elapsed time (CRS):', round(a.CRSElapsedTime / 60, 2))
        print('Elapsed time (UTC):', round((airport_time_UTC(CRSArrTime, a.Dest) - airport_time_UTC(CRSDepTime, a.Origin)).seconds / 3600, 2))
        print('\n')

time_chech([flight_top1, flight_top2])



As we can wee there is no error due to incorrect calculatinb elapsed time for different timezones of Origin and Destination. 
Let's check all outliers flights

Ok, we see that the data for the flight wiht the earliest arrival is correct. 
Let's check correctness of Elapsed times for the sample of 1000 flights

In [ ]:
# Check for correctness of CRS ET (elapsed times) for all records (rows)
from datetime import datetime

# filtering only flights that were not cancelled or diverted
not_canceled_or_diverted = (flights['Cancelled'] != 1) & (flights['Diverted'] != 1)
ordinary_flights = flights[not_canceled_or_diverted]
print('Number of not cancelled or diverted flights', len(ordinary_flights))

In [6]:
del(flights)

In [7]:
# Rename 'DayofMonth' to 'Day' for pd.to_datetime
ordinary_flights = ordinary_flights.rename(columns= {'DayofMonth': 'Day'})

In [8]:

# Create temporary field 'Minute' from 'CRSDepTime' to use with pd.to_datetime
ordinary_flights['Minute'] = ordinary_flights['CRSDepTime']
ordinary_flights['CRSDepDT'] = pd.to_datetime(ordinary_flights[['Year', 'Month', 'Day', 'Minute']])

In [9]:
ordinary_flights['CRSDepDT'] = [row.CRSDepDT.tz_localize(tz.gettz(IATAtz[row.Origin]), ambiguous=True).
                                astimezone(tz.UTC) for row in ordinary_flights[['CRSDepDT', 'Origin']].itertuples()] 

In [11]:
# Create temporary field 'Minute' from 'CRSArrTime' to use with pd.to_datetime
ordinary_flights['Minute'] = ordinary_flights['CRSArrTime']
ordinary_flights['CRSArrDT'] = pd.to_datetime(ordinary_flights[['Year', 'Month', 'Day', 'Minute']])

In [22]:
ordinary_flights['CRSArrDT'] = [row.CRSArrDT.tz_localize(tz.gettz(IATAtz[row.Dest]), ambiguous=True, 
                                                         nonexistent = 'NaT') #.astimezone(tz.UTC) 
                                for row in ordinary_flights[['CRSArrDT', 'Dest']].itertuples()]

In [ ]:
wrong_DST_filter = ordinary_flights['CRSArrDT'].isna()
print('There are', wrong_DST_filter.sum(), 'flights with wrong spring DST')

Ok, there are 41 flight with wrong spring DST time. I have saved them for future possible analysis. Let's try shift all incorrect flights forward, convert them to UTC and after that check the difference of scheduled and actual Elapsed time

In [32]:
# At first, restore the 'CRSArrDT'
ordinary_flights['Minute'] = ordinary_flights['CRSArrTime']
ordinary_flights['CRSArrDT'] = pd.to_datetime(ordinary_flights[['Year', 'Month', 'Day', 'Minute']])

# Taking only the flights with incorrect DST
flights_with_wrong_DST = ordinary_flights[wrong_DST_filter].copy()

# Shifting forward and converting to UTC
flights_with_wrong_DST['CRSArrDT'] = [row.CRSArrDT.tz_localize(tz.gettz(IATAtz[row.Dest]), ambiguous=True, 
                                                         nonexistent = 'shift_forward').astimezone(tz.UTC) 
                                for row in flights_with_wrong_DST[['CRSArrDT', 'Dest']].itertuples()]

In [ ]:
flights_with_wrong_DST[['Flight_Number_Reporting_Airline', 'CRSDepTime', 'CRSArrTime', 'CRSDepDT', 'CRSArrDT', 'CRSElapsedTime']].head(10)

In [37]:
import zipfile
source_zip = 'data/interim/csv_flight.zip'
source_path = 'csv_flight/report_'

test = load_data_from(source_zip, source_path + '2014_3.csv')

In [ ]:
Date_details = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate']
DepTime_details = ['CRSDepTime', 'DepTime']
ArrTime_details = ['CRSArrTime', 'ArrTime']
ElapsedTime_details = ['CRSElapsedTime', 'ActualElapsedTime']
CRS_details = ['CRSDepTime', 'CRSArrTime', 'CRSElapsedTime']
Route_datails = ['Origin', 'Dest']
test[(test['Flight_Number_Reporting_Airline'] == 121) & 
     (test['DayofMonth'] == 9)][Date_details + 
     CRS_details +
     Route_datails]

In [ ]:

# Create temporary field 'Minute' from 'DepTime' to use with pd.to_datetime
ordinary_flights['Minute'] = ordinary_flights['DepTime']
ordinary_flights['DepDT'] = pd.to_datetime(ordinary_flights[['Year', 'Month', 'Day', 'Minute']])

# Create temporary field 'Minute' from 'ArrTime' to use with pd.to_datetime
ordinary_flights['Minute'] = ordinary_flights['ArrTime']
ordinary_flights['ArrDT'] = pd.to_datetime(ordinary_flights[['Year', 'Month', 'Day', 'Minute']])

In [ ]:
for ind in ordinary_flights.index:
       ordinary_flights.loc[ind, 'CRSET_Diff'] = round(ordinary_flights.loc[ind, 'CRSElapsedTime'] / 60, 2) - \
                                        round((airport_time_UTC(ordinary_flights.loc[ind, 'CRSArrDT'], ordinary_flights.loc[ind, 'Dest']) - \
                                               airport_time_UTC(ordinary_flights.loc[ind, 'CRSDepDT'], ordinary_flights.loc[ind, 'Origin'])).seconds / 3600, 2)

       ordinary_flights.loc[ind, 'ActET_Diff'] = round(ordinary_flights.loc[ind, 'ActualElapsedTime'] / 60, 2) - \
                                        round((airport_time_UTC(ordinary_flights.loc[ind, 'ArrDT'], ordinary_flights.loc[ind, 'Dest']) - \
                                               airport_time_UTC(ordinary_flights.loc[ind, 'DepDT'], ordinary_flights.loc[ind, 'Origin'])).seconds / 3600, 2)

In [ ]:
ordinary_flights[:10]